In [2]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from itertools import product
from datetime import datetime, timedelta
import matplotlib.pyplot as plt
import xgboost as xgb
from sklearn.model_selection import train_test_split,train_test_split, GridSearchCV
from sklearn.metrics import accuracy_score,mean_squared_error, mean_absolute_error, r2_score,confusion_matrix
from sklearn.preprocessing import MinMaxScaler

In [2]:
import boto3
s3 = boto3.client('s3')
bucket_name = 'sagemaker-studio-905418013525-nvxe84zgs6'
file_path = 'Labrado/alllabs1000adm/alllabs1000adm0518.csv'
obj = s3.get_object(Bucket=bucket_name, Key=file_path)
rawdata = pd.read_csv(obj['Body'])rawdata=rawdata.dropna()

In [3]:
Labitemlist = rawdata['itemid'].value_counts().sort_values(ascending=False)
Labitemlist = Labitemlist.reset_index()
Labitemlist.columns = ['itemid', 'count']
len(Labitemlist)
Labitem_dict = dict(zip(Labitemlist['itemid'], Labitemlist.index))

In [100]:
Labitemlist.iloc[:10,:]

,itemid,count
0,50971,2078644
1,50983,2058719
2,50902,2042431
3,51221,2041697
4,50912,1995732
5,50882,1987133
6,50868,1981798
7,51006,1981280
8,50931,1932304
9,51265,1928290


In [40]:
unique_adm_year_day = rawdata[['hadm_id','chartyear', 'charttime_diff_days']].drop_duplicates().reset_index(drop=True)
unique_adm_year_day['rowinx'] = unique_adm_year_day.index
rawdata_index = pd.merge(rawdata, unique_adm_year_day, on=['hadm_id','chartyear', 'charttime_diff_days'], how='left')
rawdata_index['colinx'] = [Labitem_dict[itemid] for itemid in rawdata_index['itemid'].values]

In [41]:
rawdata_index.shape

(64914520, 10)

In [42]:
unique_adm_year_day.shape

(1966621, 4)

In [47]:
unique_hadm_by_year = unique_adm_year_day.groupby('chartyear')['hadm_id'].nunique()
cumulative_sum = unique_hadm_by_year.cumsum()
total_sum = cumulative_sum.iloc[-1]
split_index = cumulative_sum[cumulative_sum >= 0.8 * total_sum].index[0]
print("split year is",split_index)


split year is 2178


In [49]:
Row_Train = unique_adm_year_day['chartyear']<=split_index
Train_inx = Row_Train[Row_Train].index
Row_Test = unique_adm_year_day['chartyear']>split_index
Test_inx = Row_Test[Row_Test].index

In [9]:
npval = np.full((unique_adm_year_day.shape[0],len(Labitemlist)), np.nan)
nptime = np.full((unique_adm_year_day.shape[0],len(Labitemlist)), np.nan)
# Assign the value to npdata at the identified rows and columns
npval[rawdata_index['rowinx'], rawdata_index['colinx']] = rawdata_index['valuenum']
nptime[rawdata_index['rowinx'], rawdata_index['colinx']] = 24-rawdata_index['charthour']

In [10]:
hadm_id_indices = unique_adm_year_day.groupby('hadm_id').apply(lambda group: {'start_index': group.index.min(), 'end_index': group.index.max()+1}).to_dict()

In [11]:
npval_last = np.full((unique_adm_year_day.shape[0],len(Labitemlist)), np.nan)
nptime_last = np.full((unique_adm_year_day.shape[0],len(Labitemlist)), np.nan)
for hadm_id, val in list(hadm_id_indices.items()):
    rownum = val['end_index']-val['start_index']-1
    if rownum >0:
        npval_last[val['start_index']+1,:] = npval[val['start_index'],:]
        nptime_last[val['start_index']+1,:] =  nptime[val['start_index'],:]+24
        for i in range(val['start_index']+2, val['end_index']):
            mask_nan = np.isnan(npval[i-1,:])
            npval_last[i,:] = np.where(mask_nan, npval_last[i-1,:], npval[i-1,:])
            nptime_last[i,:] = np.where(mask_nan, nptime_last[i-1,:]+24, nptime[i-1,:]+24)

In [43]:
unique_adm_year_day.iloc[1003:1011,:]

,hadm_id,chartyear,charttime_diff_days,rowinx
1003,20006369,2111,2,1003
1004,20006377,2157,0,1004
1005,20006377,2157,1,1005
1006,20006377,2157,2,1006
1007,20006377,2157,3,1007
1008,20006377,2157,4,1008
1009,20006377,2157,5,1009
1010,20006377,2157,6,1010


In [45]:
npval

array([[  3.5, 137. , 103. , ...,   nan,   nan,   nan],
       [  3.7, 136. , 100. , ...,   nan,   nan,   nan],
       [  3.9, 138. , 102. , ...,   nan,   nan,   nan],
       ...,
       [  nan,   nan,   nan, ...,   nan,   nan,   nan],
       [  nan,   nan,   nan, ...,   nan,   nan,   nan],
       [  nan,   nan,   nan, ...,   nan,   nan,   nan]])

In [28]:
npval[1003:1009,:]

array([[  nan,   nan,   nan, ...,   nan,   nan,   nan],
       [  4. , 141. , 103. , ...,   nan,   nan,   nan],
       [  4.2, 140. , 103. , ...,   nan,   nan,   nan],
       [  4.6, 137. , 103. , ...,   nan,   nan,   nan],
       [  5.4, 136. ,  99. , ...,   nan,   nan,   nan],
       [  4.4, 133. ,  96. , ...,   nan,   nan,   nan]])

In [ ]:
np.savetxt('npval.csv', npval, delimiter=',')
file_name = "npval.csv"
file_key = "Labrado/alllabs1000adm/npval.csv"
s3.upload_file(file_name, bucket_name,file_key)

np.savetxt('nptime.csv', nptime, delimiter=',')
file_name = "nptime.csv"
file_key = "Labrado/alllabs1000adm/nptime.csv"
s3.upload_file(file_name, bucket_name,file_key)

In [46]:
np.savetxt('npval_last.csv', npval_last, delimiter=',')
file_name = "npval_last.csv"
file_key = "Labrado/alllabs1000adm/npval_last.csv"
s3.upload_file(file_name, bucket_name,file_key)

np.savetxt('nptime_last.csv', nptime_last, delimiter=',')
file_name = "nptime_last.csv"
file_key = "Labrado/alllabs1000adm/nptime_last.csv"
s3.upload_file(file_name, bucket_name,file_key)

In [ ]:
np.savetxt('Labitemlist.csv', Labitemlist, delimiter=',')
file_name = "Labitemlist.csv"
file_key = "Labrado/alllabs1000adm/Labitemlist.csv"
s3.upload_file(file_name, bucket_name,file_key)

np.savetxt('unique_adm_year_day.csv', unique_adm_year_day, delimiter=',')
file_name = "unique_adm_year_day.csv"
file_key = "Labrado/alllabs1000adm/unique_adm_year_day.csv"
s3.upload_file(file_name, bucket_name,file_key)

In [6]:
import boto3
s3 = boto3.client('s3')
bucket_name = 'sagemaker-studio-905418013525-nvxe84zgs6'


file_path = 'Labrado/alllabs1000adm/npval.csv'
obj = s3.get_object(Bucket=bucket_name, Key=file_path)
npval = pd.read_csv(obj['Body'], header=None, dtype=float)


file_path = 'Labrado/alllabs1000adm/npval_last.csv'
obj = s3.get_object(Bucket=bucket_name, Key=file_path)
npval_last = pd.read_csv(obj['Body'], header=None, dtype=float)

file_path = 'Labrado/alllabs1000adm/nptime.csv'
obj = s3.get_object(Bucket=bucket_name, Key=file_path)
nptime = pd.read_csv(obj['Body'], header=None, dtype=float)

file_path = 'Labrado/alllabs1000adm/nptime_last.csv'
obj = s3.get_object(Bucket=bucket_name, Key=file_path)
nptime_last = pd.read_csv(obj['Body'], header=None, dtype=float)

In [71]:
npval.head()

,0,1,2,3,4,5,6,7,8,9,...,365,366,367,368,369,370,371,372,373,374
0,3.5,137.0,103.0,26.5,1.1,24.0,14.0,19.0,169.0,183.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,3.7,136.0,100.0,28.1,1.2,28.0,12.0,17.0,190.0,195.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3.9,138.0,102.0,24.4,0.9,26.0,14.0,14.0,177.0,188.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,5.2,140.0,104.0,32.1,1.1,25.0,16.0,28.0,88.0,196.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4.8,141.0,106.0,30.3,2.3,24.0,11.0,26.0,165.0,157.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [9]:
file_path = 'Labrado/alllabs1000adm/Labitemlist.csv'
obj = s3.get_object(Bucket=bucket_name, Key=file_path)
Labitemlist = pd.read_csv(obj['Body'], header=None,names=['labitem', 'cnt'], dtype=int)
file_path = 'Labrado/alllabs1000adm/unique_adm_year_day.csv'
obj = s3.get_object(Bucket=bucket_name, Key=file_path)
unique_adm_year_day = pd.read_csv(obj['Body'], header=None,names=['hadm_id', 'chartyear', 'day','rowinx'], dtype=int)
unique_hadm_by_year = unique_adm_year_day.groupby('chartyear')['hadm_id'].nunique()
cumulative_sum = unique_hadm_by_year.cumsum()
total_sum = cumulative_sum.iloc[-1]
split_index = cumulative_sum[cumulative_sum >= 0.8 * total_sum].index[0]
print("split year is",split_index)
Row_Train = unique_adm_year_day['chartyear']<=split_index
Train_inx = Row_Train[Row_Train].index
Row_Test = unique_adm_year_day['chartyear']>split_index
Test_inx = Row_Test[Row_Test].index

split year is 2178


In [ ]:
LanCnt = 100

X_train = np.full((len(Train_inx),LanCnt*4), np.nan)
X_train[:,0:LanCnt*2:2] = npval.iloc[Train_inx,:LanCnt]
X_train[:,1:LanCnt*2:2] = nptime.iloc[Train_inx,:LanCnt]
X_train[:,LanCnt*2:LanCnt*4:2] = npval_last.iloc[Train_inx,:LanCnt]
X_train[:,LanCnt*2+1:LanCnt*4:2] = nptime_last.iloc[Train_inx,:LanCnt]

X_test = np.full((len(Test_inx),LanCnt*4), np.nan)
X_test[:,0:LanCnt*2:2] = npval.iloc[Test_inx,:LanCnt]
X_test[:,1:LanCnt*2:2] = nptime.iloc[Test_inx,:LanCnt]
X_test[:,LanCnt*2:LanCnt*4:2] = npval_last.iloc[Test_inx,:LanCnt]
X_test[:,LanCnt*2+1:LanCnt*4:2] = nptime_last.iloc[Test_inx,:LanCnt]

X_train_df = pd.DataFrame(X_train)
X_test_df = pd.DataFrame(X_test)

# Optionally, you can assign column names to the DataFrames
# Generate column names for each part of the DataFrame
col_names = []
for i in range(LanCnt):
    col_names.append(f'npval_{Labitemlist[i,0]}')
    col_names.append(f'nptime_{Labitemlist[i,0]}')
for i in range(LanCnt):
    col_names.append(f'npval_last_{Labitemlist[i,0]}')
    col_names.append(f'nptime_last_{Labitemlist[i,0]}')

X_train_df.to_csv('X_train.csv', index=False)


file_name = "X_train.csv"
file_key = "Labrado/alllabs1000adm/X_train.csv"
s3.upload_file(file_name, bucket_name,file_key)

X_test_df.to_csv('X_test.csv', index=False)
file_name = "X_test.csv"
file_key = "Labrado/alllabs1000adm/X_test.csv"
s3.upload_file(file_name, bucket_name,file_key)

In [ ]:
ItemID = 2
columns_to_remove = [ItemID*2, ItemID*2+1]  # Assuming you want to remove the second and third columns (0-indexed)

y_train = X_train[:,ItemID*2]
X_train = X_train[~np.isnan(y_train),:]
y_train = y_train[~np.isnan(y_train)]

y_test = X_test[:,ItemID*2]
X_test = X_test[~np.isnan(y_test),:]
y_test = y_test[~np.isnan(y_test)]

X_train = np.delete(X_train, columns_to_remove, axis=1)
X_test = np.delete(X_test, columns_to_remove, axis=1)

In [86]:
def train_xgboost_model(X_train, X_test, y_train, y_test):
    # Split the data into training and testing sets

    # Define the XGBoost model
    model = xgb.XGBRegressor(objective="reg:squarederror")

    # Define the hyperparameter grid for grid search
    param_grid = {
        'learning_rate': [0.01, 0.1, 0.2],
        'max_depth': [3, 4, 5],
        'n_estimators': [50, 100, 200],
    }

    # Use GridSearchCV to find the best hyperparameters
    grid_search = GridSearchCV(model, param_grid, scoring='neg_mean_squared_error', cv=3)
    grid_search.fit(X_train, y_train)

    # Get the best model from the grid search
    best_model = grid_search.best_estimator_

    # Make predictions on the test set using the best model
    predictions = best_model.predict(X_test)

    # Calculate evaluation metrics
    mse = mean_squared_error(y_test, predictions)
    mae = mean_absolute_error(y_test, predictions)
    r2 = r2_score(y_test, predictions)
    std_dev = np.std(y_test - predictions)  # Standard deviation of residuals

    return {
        "y_test": y_test,
        "predictions": predictions,
        "mse": mse,
        "mae": mae,
        "r2": r2,
        "std_dev": std_dev,
        "best_params": grid_search.best_params_
    }

In [ ]:
print("Current time:", datetime.now())
model_results = train_xgboost_model(X_train, X_test, y_train, y_test)
print("Current time:", datetime.now())
print(model_results)

Current time: 2024-05-20 22:12:11.431524
Current time: 2024-05-21 00:01:54.058934
{'y_test': array([102., 103., 103., ..., 110., 101., 100.]), 'predictions': array([101.974, 103.122, 103.336, ..., 110.294, 101.126,  99.847],
      dtype=float32), 'mse': 0.7537429154574968, 'mae': 0.5347291202779194, 'r2': 0.9754317327054305, 'std_dev': 0.86813829526865, 'best_params': {'learning_rate': 0.2, 'max_depth': 5, 'n_estimators': 200}}


In [ ]:
import numpy as np
from sklearn.metrics import confusion_matrix

def mse_r2_confusion_matrix(ground_truth, predictions, normal_range):
    """
    Calculate the Mean Squared Error (MSE), R-squared value (R2), and confusion matrix for two arrays, ignoring NaN values.
    The confusion matrix will be based on three categories: lower, normal, and higher.

    Parameters:
    ground_truth (numpy.ndarray): Ground truth array.
    predictions (numpy.ndarray): Predicted values array.
    normal_range (tuple): Normal range as a tuple (min, max).

    Returns:
    tuple: Three elements:
        - float: MSE value.
        - float: R-squared value.
        - numpy.ndarray: Confusion matrix.
    """
    # Ensure input arrays are numpy arrays
    ground_truth = np.asarray(ground_truth)
    predictions = np.asarray(predictions)

    if ground_truth.shape != predictions.shape:
        raise ValueError("The shapes of the ground truth and prediction arrays must be the same.")

    # Handle NaN values by masking
    mask = np.isnan(ground_truth) | np.isnan(predictions)
    gt = ground_truth[~mask]
    pred = predictions[~mask]

    # Calculate MSE
    mse = np.mean((gt - pred) ** 2)

    # Calculate R-squared
    mean_gt = np.mean(gt)
    tss = np.sum((gt - mean_gt) ** 2)
    rss = np.sum((gt - pred) ** 2)
    r2 = 1 - rss / tss

    # Normal range handling
    normal_min, normal_max = normal_range
    gt_category = np.where(gt < normal_min, 0, np.where(gt > normal_max, 2, 1))
    pred_category = np.where(pred < normal_min, 0, np.where(pred > normal_max, 2, 1))

    # Calculate confusion matrix
    cm = confusion_matrix(gt_category, pred_category, labels=[0, 1, 2])

    return mse, r2, cm

In [ ]:
normal_ranges[ItemID]

(98, 106)

In [ ]:
normal_ranges = [(3.5,5),(136,145),(98,106),(37,50),(0.7,1.3),(23,28),(7,13),(8,20),(0,200),(150,450)]
np.set_printoptions(precision=3, suppress=True)
print(X_test.shape)
print(normal_ranges[ItemID])


y_pred = model_results['predictions']
mse, r2, cm = mse_r2_confusion_matrix(y_test, y_pred, normal_ranges[ItemID])

print("Overall Xgboost MSE:", round(mse,3))
print("Overall Xgboost R-squared:", round(r2,3))
print("Overall Xgboost Confusion matrix:\n", cm)

y_last = X_test[:,ItemID*2+LanCnt*2-2]
valid_pos = ~np.isnan(y_last)
print('Test with the last value',len(y_last[valid_pos]))
mse, r2, cm = mse_r2_confusion_matrix(y_test, y_last, normal_ranges[ItemID])
print("last value prediction MSE per column:", round(mse,3))
print("last value prediction R-squared per column:", round(r2,3))
print("last value prediction Confusion matrix:\n", cm)


valid_pos = ~np.isnan(y_last)
mse, r2, cm = mse_r2_confusion_matrix(y_test[valid_pos], y_pred[valid_pos], normal_ranges[ItemID])
print("XGBoost for those with last value MSE :", round(mse,3))
print("XGBoost for those with last value R-squared:", round(r2,3))
print("XGBoost for those with last value Confusion matrix:\n", cm)

(331789, 398)
(98, 106)
Overall Xgboost MSE: 0.754
Overall Xgboost R-squared: 0.975
Overall Xgboost Confusion matrix:
 [[ 59695   1875      8]
 [  8587 188238  10357]
 [     8   1958  61063]]
Test with the last value 262513
last value prediction MSE per column: 8.909
last value prediction R-squared per column: 0.719
last value prediction Confusion matrix:
 [[ 39122  12455    155]
 [ 12917 131962  16472]
 [   188  14743  34499]]
XGBoost for those with last value MSE : 0.626
XGBoost for those with last value R-squared: 0.98
XGBoost for those with last value Confusion matrix:
 [[ 50619   1109      4]
 [  6810 146416   8125]
 [     6   1326  48098]]
